In [1]:
 '''
Author information:
Joey R. Muffoletto
University of Texas at Austin
Autonomous Systems Group
jrmuff@utexas.edu
'''

'\nAuthor information:\nJoey R. Muffoletto\nUniversity of Texas at Austin\nAutonomous Systems Group\njrmuff@utexas.edu\n'

In [2]:
import realtime_manager as rm 
import graph_manager as gm
import reworked_graph as rg
import copy
import os
os.environ["PATH"] += os.pathsep + 'C:/Program Files/Graphviz 2.44.1/bin'

c:\users\joey\appdata\local\programs\python\python37\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
c:\users\joey\appdata\local\programs\python\python37\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
c:\users\joey\appdata\local\programs\python\python37\lib\site-packages\numpy\.libs\libopenblas.TXA6YQSD3GCQQC22GEQ54J2UDCXDXHWN.gfortran-win_amd64.dll
  stacklevel=1)


In [3]:
'''
manager configurations

traffic test globals
'''
NUM_TOWERS = 5
TAU = 2

LOW_TRAFFIC_MULTIPLIER = 1
HIGH_TRAFFIC_MULTIPLIER = 4

MIN_LOW_TRAFFIC = 0
MAX_LOW_TRAFFIC = LOW_TRAFFIC_MULTIPLIER * NUM_TOWERS

MIN_HIGH_TRAFFIC = MAX_LOW_TRAFFIC
MAX_HIGH_TRAFFIC = HIGH_TRAFFIC_MULTIPLIER * NUM_TOWERS

MIN_TTL = 2
MAX_TTL = 6

DEFAULT_EMPTY_STATE = rg.State((),(),{"0" : 6})
rm.configure_realtime(tau=TAU, override_default_empty_state=DEFAULT_EMPTY_STATE)

HIGH_TRAFFIC_FREQUENCY = .2 # use rand.random() = [0.0, 1.0], or, just add this value until = 1 then reset

NUM_TIME_STEPS = 40



In [4]:
'''
Input generation functions

low traffic is some random number between min_low_traffic and max_low_traffic, high traffic is the same with high

choose a random tower index everytime we add one

high traffic occurs everytime HIGH_TRAFFIC_FREQUENCY * TIME_STEPS is a whole number

TTL is a random number between MIN_TTL and MAX_TTL
'''
import random
random.seed(10)
def generate_traffic(min_traffic, max_traffic):
    additional_requests_dict = dict()
#     counter = 0
    requests_to_add = random.randint(min_traffic, max_traffic)
    for i in range(requests_to_add):
#         if counter == NUM_TOWERS:
#             counter = 0
#         tower_to_add_to = counter
#         counter+=1
        tower_to_add_to = random.randint(0, NUM_TOWERS-1) # NUM_TOWERS is OOB
        print(tower_to_add_to)
        request_to_add = ('no_pref', random.randint(MIN_TTL, MAX_TTL))
#         request_to_add = ('no_pref', 5)
        if tower_to_add_to in additional_requests_dict:
            additional_requests_dict[tower_to_add_to].append(request_to_add)
        else:
            additional_requests_dict[tower_to_add_to] = [request_to_add]
    return additional_requests_dict
            
def generate_low_traffic():
    return generate_traffic(MIN_LOW_TRAFFIC, MAX_LOW_TRAFFIC)
    
def generate_high_traffic():
    return generate_traffic(MIN_HIGH_TRAFFIC, MAX_HIGH_TRAFFIC)

In [5]:
'''
Input Generation
'''
input = []
high_traffic_counter = 0
for time in range(NUM_TIME_STEPS):
    if high_traffic_counter == 1:
        high_traffic_counter = 0
        input.append([generate_high_traffic()])
    else:
        high_traffic_counter += HIGH_TRAFFIC_FREQUENCY
        input.append([generate_low_traffic()])
        
initial_system = [copy.deepcopy(gm.return_tower(0, 1, [],[6])) for i in range(NUM_TOWERS)]

0
3
0
3
1
4
0
2
1
2
3
2
1
2
3
3
0
0
1
2
2
2
3
3
4
4
1
1
0
2
0
0
3
4
0
1
2
3
2
4
0
3
1
3
3
0
0
2
0
3
4
3
1
2
3
3
1
3
1
4
4
3
4
0
3
0
0
0
3
3
4
2
3
4
1
2
0
4
3
0
3
2
0
4
2
3
4
1
0
3
2
4
3
1
4
0
3
3
1
3
0
2
3
2
2
3
2
3
2
3
4
0
2
1
3
0
2
2
0
4
2
1
2
4
1
0
1
4
4
3
2
4
0
0
2
2
1
0
1
4
0
3
0
3
4
3
1
3
4
3
1
3
0
1
3
4
0
0
1
4
4
1
0
4
0
3


In [6]:
'''
Input statistics:
'''
requests_per_tower = [0 for i in range(NUM_TOWERS)]
for time_step in range(len(input)):
    for i in range(NUM_TOWERS):
        if i in input[time_step][0]:
            requests_per_tower[i]+=len(input[time_step][0][i])
for i in range(NUM_TOWERS):
    print("tower " + str(i) + "has " + str(requests_per_tower[i]) + " requests")

tower 0has 37 requests
tower 1has 26 requests
tower 2has 30 requests
tower 3has 44 requests
tower 4has 29 requests


In [7]:
'''
INPUT SAVING (DO NOT RE-RUN WILLY NILLY MY FRIEND)
'''
input_copy = copy.deepcopy(input)
for _dict in input_copy:
    print(_dict)

initial_system_copy = copy.deepcopy(initial_system)

[{0: [('no_pref', 5), ('no_pref', 3)], 3: [('no_pref', 6), ('no_pref', 5)]}]
[{1: [('no_pref', 2)], 4: [('no_pref', 5)]}]
[{0: [('no_pref', 3)], 2: [('no_pref', 2)]}]
[{1: [('no_pref', 6)], 2: [('no_pref', 5)], 3: [('no_pref', 4)]}]
[{2: [('no_pref', 5), ('no_pref', 3)], 1: [('no_pref', 4)], 3: [('no_pref', 3), ('no_pref', 6)]}]
[{0: [('no_pref', 6), ('no_pref', 3), ('no_pref', 5), ('no_pref', 6), ('no_pref', 3)], 1: [('no_pref', 3), ('no_pref', 5), ('no_pref', 2)], 2: [('no_pref', 6), ('no_pref', 3), ('no_pref', 6), ('no_pref', 6)], 3: [('no_pref', 5), ('no_pref', 2), ('no_pref', 6)], 4: [('no_pref', 4), ('no_pref', 3)]}]
[{4: [('no_pref', 3)], 0: [('no_pref', 2)]}]
[{1: [('no_pref', 3)], 2: [('no_pref', 5)], 3: [('no_pref', 5)]}]
[{2: [('no_pref', 3)]}]
[{4: [('no_pref', 3)], 0: [('no_pref', 4)], 3: [('no_pref', 4)], 1: [('no_pref', 3)]}]
[{3: [('no_pref', 4)]}]
[{3: [('no_pref', 3), ('no_pref', 5), ('no_pref', 5), ('no_pref', 6), ('no_pref', 2), ('no_pref', 5)], 0: [('no_pref', 6), 

In [8]:
'''
data collection
'''
_completed_states, _timing_info = rm.main_loop(initial_system_copy, input_copy)


minimized_traces[[State currently has requests :: () Time states of :: () Port states of :: {'0': 6} Labels :: ('VALID', 'FINISH')  
], [State currently has requests :: () Time states of :: () Port states of :: {'0': 6} Labels :: ('VALID', 'FINISH')  
], [State currently has requests :: () Time states of :: () Port states of :: {'0': 6} Labels :: ('VALID', 'FINISH')  
], [State currently has requests :: () Time states of :: () Port states of :: {'0': 6} Labels :: ('VALID', 'FINISH')  
], [State currently has requests :: () Time states of :: () Port states of :: {'0': 6} Labels :: ('VALID', 'FINISH')  
]]
[{0: [('no_pref', 5), ('no_pref', 3)], 3: [('no_pref', 6), ('no_pref', 5)]}]
additional requests for this tower = [('no_pref', 5), ('no_pref', 3)]
last request_vector = 0
last request_vector = 0
Tau trace = [State currently has requests :: ('no_pref', 'no_pref') Time states of :: (5, 3) Port states of :: {'0': 6} Labels :: ('VALID',)  
, State currently has requests :: ('no_pref',) Tim

Tau trace = [State currently has requests :: ('no_pref', 'no_pref', 'no_pref', 'no_pref', 'no_pref', 'no_pref') Time states of :: (2, 4, 4, 5, 1, 4) Port states of :: {'0': -3} Labels :: ('VALID',)  
, State currently has requests :: ('no_pref', 'no_pref', 'no_pref', 'no_pref', 'no_pref') Time states of :: (1, 3, 3, 4, 3) Port states of :: {'0': -4} Labels :: ('VALID', 'OVERFLOWED_PORT')  
, State currently has requests :: ('no_pref', 'no_pref', 'no_pref', 'no_pref') Time states of :: (2, 2, 3, 2) Port states of :: {'0': -5} Labels :: ('VALID', 'OVERFLOWED_PORT')  
, State currently has requests :: ('no_pref', 'no_pref', 'no_pref') Time states of :: (1, 2, 1) Port states of :: {'0': -6} Labels :: ('VALID', 'OVERFLOWED_PORT')  
, State currently has requests :: ('no_pref', 'no_pref') Time states of :: (0, 1) Port states of :: {'0': -7} Labels :: ('VALID', 'OVERFLOWED_PORT')  
, State currently has requests :: ('no_pref',) Time states of :: (0,) Port states of :: {'0': -8} Labels :: ('VA

Tau trace = [State currently has requests :: ('no_pref', 'no_pref', 'no_pref', 'no_pref') Time states of :: (5, 6, 2, 3) Port states of :: {'0': 6} Labels :: ('VALID',)  
, State currently has requests :: ('no_pref', 'no_pref', 'no_pref') Time states of :: (4, 5, 2) Port states of :: {'0': 5} Labels :: ('VALID',)  
, State currently has requests :: ('no_pref', 'no_pref') Time states of :: (3, 4) Port states of :: {'0': 4} Labels :: ('VALID',)  
, State currently has requests :: ('no_pref',) Time states of :: (3,) Port states of :: {'0': 3} Labels :: ('VALID',)  
, State currently has requests :: () Time states of :: () Port states of :: {'0': 2} Labels :: ('VALID', 'FINISH')  
]
Full trace including new requests = [State currently has requests :: ('no_pref', 'no_pref', 'no_pref', 'no_pref') Time states of :: (5, 6, 2, 3) Port states of :: {'0': 6} Labels :: ('VALID',)  
, State currently has requests :: ('no_pref', 'no_pref', 'no_pref') Time states of :: (4, 5, 2) Port states of :: {'0

Tau trace = [State currently has requests :: ('no_pref', 'no_pref', 'no_pref', 'no_pref', 'no_pref', 'no_pref', 'no_pref') Time states of :: (-1, 1, 2, 1, 1, 4, 2) Port states of :: {'0': -18} Labels :: ()  
, State currently has requests :: ('no_pref', 'no_pref', 'no_pref', 'no_pref', 'no_pref', 'no_pref') Time states of :: (0, 1, 0, 0, 3, 1) Port states of :: {'0': -19} Labels :: ('OVERFLOWED_PORT',)  
, State currently has requests :: ('no_pref', 'no_pref', 'no_pref', 'no_pref', 'no_pref') Time states of :: (0, -1, -1, 2, 0) Port states of :: {'0': -20} Labels :: ('OVERFLOWED_PORT',)  
, State currently has requests :: ('no_pref', 'no_pref', 'no_pref', 'no_pref') Time states of :: (-1, -1, 1, -1) Port states of :: {'0': -21} Labels :: ('OVERFLOWED_PORT',)  
, State currently has requests :: ('no_pref', 'no_pref', 'no_pref') Time states of :: (-1, -1, 0) Port states of :: {'0': -22} Labels :: ('OVERFLOWED_PORT',)  
, State currently has requests :: ('no_pref', 'no_pref') Time states 

Full trace including new requests = [State currently has requests :: ('no_pref',) Time states of :: (5,) Port states of :: {'0': 6} Labels :: ('VALID',)  
, State currently has requests :: () Time states of :: () Port states of :: {'0': 5} Labels :: ('VALID', 'FINISH')  
]
additional requests for this tower = [('no_pref', 6)]
last request_vector = 0
Tau trace = [State currently has requests :: ('no_pref',) Time states of :: (6,) Port states of :: {'0': 6} Labels :: ('VALID',)  
, State currently has requests :: () Time states of :: () Port states of :: {'0': 5} Labels :: ('VALID', 'FINISH')  
]
Full trace including new requests = [State currently has requests :: ('no_pref',) Time states of :: (6,) Port states of :: {'0': 6} Labels :: ('VALID',)  
, State currently has requests :: () Time states of :: () Port states of :: {'0': 5} Labels :: ('VALID', 'FINISH')  
]
completed traces = [[State currently has requests :: ('no_pref', 'no_pref') Time states of :: (5, 3) Port states of :: {'0':

]
Full trace including new requests = [State currently has requests :: ('no_pref', 'no_pref') Time states of :: (-1, -1) Port states of :: {'0': -26} Labels :: ('OVERFLOWED_PORT',)  
, State currently has requests :: ('no_pref', 'no_pref') Time states of :: (-1, 4) Port states of :: {'0': -27} Labels :: ()  
, State currently has requests :: ('no_pref', 'no_pref', 'no_pref', 'no_pref', 'no_pref', 'no_pref', 'no_pref') Time states of :: (3, 3, 4, 2, 3, 1, 2) Port states of :: {'0': -28} Labels :: ('VALID',)  
, State currently has requests :: ('no_pref', 'no_pref', 'no_pref', 'no_pref', 'no_pref', 'no_pref') Time states of :: (2, 2, 3, 1, 2, 1) Port states of :: {'0': -29} Labels :: ('VALID', 'OVERFLOWED_PORT')  
, State currently has requests :: ('no_pref', 'no_pref', 'no_pref', 'no_pref', 'no_pref') Time states of :: (1, 1, 2, 0, 1) Port states of :: {'0': -30} Labels :: ('VALID', 'OVERFLOWED_PORT')  
, State currently has requests :: ('no_pref', 'no_pref', 'no_pref', 'no_pref') Time 

In [9]:
'''
data buffer (DO NOT RE-RUN WILLY-NILLY)
'''
completed_states = copy.deepcopy(_completed_states)
timing_info = copy.deepcopy(_timing_info)




In [10]:
'''
process data buffer
'''
cum_num_reqs = [0 for i in range(len(completed_states))]
for index,com in enumerate(completed_states):
    for state in com:
        print(state.labels)
        if("VALID" in state.labels):
            cum_num_reqs[index] += 1
    print("\n\n NEW TOWER")
print (cum_num_reqs)
# mvp_output_per_tower
mvp_output_per_tower = rm.get_mvp_output(completed_states)
for output in mvp_output_per_tower:
    gm.print_formatted_cost(output[0],format_override=True)
#     output[3].plot()
#     gm.print_formatted_trace_path(output[1])

print (cum_num_reqs)
for step in timing_info:
    print(step*1000)
    

('VALID',)
('VALID',)
('VALID',)
('VALID', 'FINISH')
('VALID', 'FINISH')
('VALID',)
('VALID',)
('VALID',)
('VALID',)
('VALID',)
('VALID',)
('VALID',)
('VALID',)
('VALID', 'OVERFLOWED_PORT')
('VALID', 'OVERFLOWED_PORT')
('VALID',)
('VALID', 'OVERFLOWED_PORT')
('VALID',)
('VALID', 'OVERFLOWED_PORT')
('VALID',)
('VALID', 'OVERFLOWED_PORT')
('VALID',)
('VALID', 'OVERFLOWED_PORT')
('VALID',)
('VALID', 'OVERFLOWED_PORT')
('VALID', 'FINISH', 'OVERFLOWED_PORT')
('VALID',)
('VALID',)
('VALID',)
('VALID',)
('VALID', 'FINISH')
('VALID',)
('VALID',)
('VALID',)
('VALID',)
('VALID',)
('VALID',)
('VALID',)
('VALID', 'OVERFLOWED_PORT')
('VALID',)
('VALID', 'OVERFLOWED_PORT')
('VALID', 'FINISH', 'OVERFLOWED_PORT')
('VALID', 'FINISH')
('VALID', 'FINISH')
('VALID', 'FINISH')
('VALID', 'FINISH')


 NEW TOWER
('VALID', 'FINISH')
('VALID',)
('VALID', 'FINISH')
('VALID',)
('VALID',)
('VALID',)
('VALID',)
('VALID',)
('VALID',)
('VALID',)
('VALID', 'FINISH', 'OVERFLOWED_PORT')
('VALID',)
('VALID',)
('VALID',)
